In [23]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, State, ctx
from dash.exceptions import PreventUpdate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import dash
from dash import dcc, html, Input, Output

import plotly.express as px

In [24]:
# Data loading and preparation
df = pd.read_csv('updated_combined_sales_plants.csv')


In [25]:
df

,PlantName,PlantCity,PlantPostalCode,PlantStreet,Plant,PlantType,RequestedDeliveryDate,DeliveryDate,OrderQuantity,SalesOrderCreationDate,DeliveryDelay,Latitude,Longitude
0,Antwerp DC,Antwerp,2030,Scheldelaan,ANT2,Distribution Center,2024-11-15,2024-11-15,26,2024-10-08,0 days,51.2194,4.4025
1,Wrocław DC,Wrocław,54-202,Legnicka,WRO2,Distribution Center,2023-06-15,2023-06-12,20,2023-05-27,-3 days,51.1079,17.0385
2,Wrocław DC,Wrocław,54-202,Legnicka,WRO2,Distribution Center,2022-04-15,2022-04-15,13,2022-03-15,0 days,51.1079,17.0385
3,Antwerp DC,Antwerp,2030,Scheldelaan,ANT2,Distribution Center,2022-04-15,2022-04-12,21,2022-03-27,-3 days,51.2194,4.4025
4,Göteborg DC,Göteborg,418 78,Assar Gabrielssons väg,GOT2,Distribution Center,2023-02-23,2023-02-24,16,2023-01-22,1 days,57.7089,11.9746
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Göteborg DC,Göteborg,418 78,Assar Gabrielssons väg,GOT2,Distribution Center,2024-12-17,2024-12-13,13,2024-11-12,-4 days,57.7089,11.9746
99996,Antwerp DC,Antwerp,2030,Scheldelaan,ANT2,Distribution Center,2022-06-13,2022-06-09,17,2022-05-21,-4 days,51.2194,4.4025
99997,Göteborg DC,Göteborg,418 78,Assar Gabrielssons väg,GOT2,Distribution Center,2022-12-23,2022-12-22,17,2022-12-22,-1 days,57.7089,11.9746
99998,Antwerp DC,Antwerp,2030,Scheldelaan,ANT2,Distribution Center,2022-07-12,2022-07-08,20,2022-06-03,-4 days,51.2194,4.4025


In [26]:

df['RequestedDeliveryDate'] = pd.to_datetime(df['RequestedDeliveryDate'])
df['DeliveryDate'] = pd.to_datetime(df['DeliveryDate'])
df['SalesOrderCreationDate'] = pd.to_datetime(df['SalesOrderCreationDate'])
df['DeliveryDelay'] = pd.to_numeric(df['DeliveryDelay'].str.replace(' days', ''))


In [27]:

app = dash.Dash(__name__)

In [28]:

app.layout = html.Div([
    html.H1("Supply Chain Monitoring Dashboard"),
    dcc.Dropdown(
        id='plant-type-dropdown',
        options=[{'label': plant, 'value': plant} for plant in df['PlantType'].unique()],
        value='Distribution Center',
        multi=True,
        placeholder="Select Plant Type"
    ),
    dcc.Graph(id='map-plot'),
    dcc.Graph(id='delay-histogram'),
    dcc.Graph(id='quantity-line-chart')
])


In [29]:

@app.callback(
    [Output('map-plot', 'figure'),
     Output('delay-histogram', 'figure'),
     Output('quantity-line-chart', 'figure')],
    [Input('plant-type-dropdown', 'value')]
)

def update_charts(selected_plant_types):
    if not selected_plant_types:
        filtered_data = data
    else:
        filtered_data = data[data['PlantType'].isin(selected_plant_types)]
    
  

In [30]:

app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Supply Chain Monitoring Dashboard"),
    dcc.Dropdown(
        id='plant-type-dropdown',
        options=[{'label': plant, 'value': plant} for plant in df['PlantType'].unique()],
        value=['Distribution Center'],
        multi=True,
        placeholder="Select Plant Type"
    ),
    dcc.Graph(id='map-plot'),
    dcc.Graph(id='delay-histogram'),
    dcc.Graph(id='quantity-line-chart')
])

# Combined callback to update the figures based on user interactions
@app.callback(
    [Output('map-plot', 'figure'),
     Output('delay-histogram', 'figure'),
     Output('quantity-line-chart', 'figure')],
    [Input('plant-type-dropdown', 'value')]
)
def update_charts(selected_plant_types):
    # Filter data based on selected plant types
    if not selected_plant_types:
        filtered_data = df
    else:
        filtered_data = df[df['PlantType'].isin(selected_plant_types)]
    
    # Map plot
    map_fig = px.scatter_mapbox(
        filtered_data,
        lat="Latitude",
        lon="Longitude",
        hover_name="PlantName",
        hover_data={
            "PlantCity": True,
            "OrderQuantity": True,
            "DeliveryDelay": True
        },
        color="DeliveryDelay",
        size="OrderQuantity",
        color_continuous_scale=px.colors.cyclical.IceFire,
        size_max=15,
        zoom=3,
        mapbox_style="carto-positron",
        title="Distribution Centers and Delivery Delays"
    )

    # Delivery delay histogram
    delay_hist = px.histogram(
        filtered_data,
        x='DeliveryDelay',
        nbins=30,
        title="Distribution of Delivery Delays",
        labels={'DeliveryDelay': 'Delivery Delay (days)'}
    )

    # Order quantity over time line chart
    quantity_line = px.line(
        filtered_data,
        x='RequestedDeliveryDate',
        y='OrderQuantity',
        color='PlantName',
        title="Order Quantity Over Time",
        labels={'RequestedDeliveryDate': 'Date', 'OrderQuantity': 'Order Quantity'}
    )

    return map_fig, delay_hist, quantity_line

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [31]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import dash_leaflet as dl
import dash_leaflet.express as dlx

ModuleNotFoundError: No module named 'dash_leaflet'

In [ ]:

app = dash.Dash(__name__)


def create_geojson(df):
    features = []
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row['Longitude'], row['Latitude']]
            },
            "properties": {
                "plant_name": row['PlantName'],
                "plant_city": row['PlantCity'],
                "order_quantity": row['OrderQuantity'],
                "delivery_delay": row['DeliveryDelay'],
                "plant_type": row['PlantType']
            }
        }
        features.append(feature)
    return {
        "type": "FeatureCollection",
        "features": features
    }

geojson_data = create_geojson(df)


app.layout = html.Div([
    html.H1("Supply Chain Monitoring Dashboard"),
    dcc.Dropdown(
        id='plant-type-dropdown',
        options=[{'label': plant, 'value': plant} for plant in df['PlantType'].unique()],
        value=['Distribution Center'],
        multi=True,
        placeholder="Select Plant Type"
    ),
    dl.Map(center=[50, 10], zoom=4, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.LayerGroup(id="layer"),
        dl.LayerControl(),
    ], style={'width': '100%', 'height': '500px'}),
    dcc.Graph(id='delay-histogram'),
    dcc.Graph(id='quantity-line-chart')
])


@app.callback(
    [Output('layer', 'children'),
     Output('delay-histogram', 'figure'),
     Output('quantity-line-chart', 'figure')],
    [Input('plant-type-dropdown', 'value')]
)
def update_charts(selected_plant_types):

    if not selected_plant_types:
        filtered_data = data
    else:
        filtered_data = df[df['PlantType'].isin(selected_plant_types)]
    
   
    geojson_data = create_geojson(filtered_data)

    
    markers = [
        dl.Marker(position=[feature['geometry']['coordinates'][1], feature['geometry']['coordinates'][0]], children=[
            dl.Tooltip(feature['properties']['plant_name']),
            dl.Popup([
                html.H3(feature['properties']['plant_name']),
                html.P(f"City: {feature['properties']['plant_city']}"),
                html.P(f"Order Quantity: {feature['properties']['order_quantity']}"),
                html.P(f"Delivery Delay: {feature['properties']['delivery_delay']} days")
            ])
        ]) for feature in geojson_data['features']
    ]

   
    delay_hist = px.histogram(
        filtered_data,
        x='DeliveryDelay',
        nbins=30,
        title="Distribution of Delivery Delays",
        labels={'DeliveryDelay': 'Delivery Delay (days)'}
    )

    
    quantity_line = px.line(
        filtered_data,
        x='RequestedDeliveryDate',
        y='OrderQuantity',
        color='PlantName',
        title="Order Quantity Over Time",
        labels={'RequestedDeliveryDate': 'Date', 'OrderQuantity': 'Order Quantity'}
    )

    return markers, delay_hist, quantity_line


if __name__ == '__main__':
    app.run_server(debug=True)

In [32]:
# 3

In [33]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [34]:

data = pd.read_csv('updated_combined_sales_plants.csv')

In [35]:

data['RequestedDeliveryDate'] = pd.to_datetime(data['RequestedDeliveryDate'])
data['DeliveryDate'] = pd.to_datetime(data['DeliveryDate'])
data['SalesOrderCreationDate'] = pd.to_datetime(data['SalesOrderCreationDate'])
data['DeliveryDelay'] = pd.to_numeric(data['DeliveryDelay'].str.replace(' days', ''))



In [36]:


app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Supply Chain Monitoring Dashboard"),
    dcc.Dropdown(
        id='plant-type-dropdown',
        options=[{'label': plant, 'value': plant} for plant in data['PlantType'].unique()],
        value=['Distribution Center'],
        multi=True,
        placeholder="Select Plant Type"
    ),
    dcc.Graph(id='map-plot'),
    dcc.Graph(id='delay-histogram'),
    dcc.Graph(id='quantity-line-chart')
])


@app.callback(
    [Output('map-plot', 'figure'),
     Output('delay-histogram', 'figure'),
     Output('quantity-line-chart', 'figure')],
    [Input('plant-type-dropdown', 'value')]
)
def update_charts(selected_plant_types):
    
    if not selected_plant_types:
        filtered_data = data
    else:
        filtered_data = data[data['PlantType'].isin(selected_plant_types)]
    
  
    map_fig = px.scatter_mapbox(
        filtered_data,
        lat="Latitude",
        lon="Longitude",
        hover_name="PlantName",
        hover_data={
            "PlantCity": True,
            "OrderQuantity": True,
            "DeliveryDelay": True
        },
        color="DeliveryDelay",
        size="OrderQuantity",
        color_continuous_scale=px.colors.cyclical.IceFire,
        size_max=15,
        zoom=3,
        mapbox_style="carto-positron",
        title="Distribution Centers and Delivery Delays"
    )

   
    delay_hist = px.histogram(
        filtered_data,
        x='DeliveryDelay',
        nbins=30,
        title="Distribution of Delivery Delays",
        labels={'DeliveryDelay': 'Delivery Delay (days)'}
    )

    
    quantity_line = px.line(
        filtered_data,
        x='RequestedDeliveryDate',
        y='OrderQuantity',
        color='PlantName',
        title="Order Quantity Over Time",
        labels={'RequestedDeliveryDate': 'Date', 'OrderQuantity': 'Order Quantity'}
    )

    return map_fig, delay_hist, quantity_line

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



In [37]:

data['RequestedDeliveryDate'] = pd.to_datetime(data['RequestedDeliveryDate'])
data['DeliveryDate'] = pd.to_datetime(data['DeliveryDate'])
data['SalesOrderCreationDate'] = pd.to_datetime(data['SalesOrderCreationDate'])
data['DeliveryDelay'] = pd.to_numeric(data['DeliveryDelay'].str.replace(' days', ''))

app = dash.Dash(__name__)


app.layout = html.Div([
    html.H1("Supply Chain Monitoring Dashboard"),
    dcc.Dropdown(
        id='plant-type-dropdown',
        options=[{'label': plant, 'value': plant} for plant in data['PlantType'].unique()],
        value=['Distribution Center'],
        multi=True,
        placeholder="Select Plant Type"
    ),
    dcc.Graph(id='map-plot'),
    dcc.Graph(id='delay-histogram'),
    dcc.Graph(id='quantity-line-chart')
])


@app.callback(
    [Output('map-plot', 'figure'),
     Output('delay-histogram', 'figure'),
     Output('quantity-line-chart', 'figure')],
    [Input('plant-type-dropdown', 'value')]
)
def update_charts(selected_plant_types):
    
    if not selected_plant_types:
        filtered_data = data
    else:
        filtered_data = data[data['PlantType'].isin(selected_plant_types)]
    
    map_fig = go.Figure()

    map_fig.add_trace(go.Scattermapbox(
        lat=filtered_data['Latitude'],
        lon=filtered_data['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=filtered_data['OrderQuantity'] / 10,
            color=filtered_data['DeliveryDelay'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Delivery Delay (days)")
        ),
        text=filtered_data.apply(lambda row: f"{row['PlantName']}<br>City: {row['PlantCity']}<br>Order Quantity: {row['OrderQuantity']}<br>Delivery Delay: {row['DeliveryDelay']} days", axis=1),
        hoverinfo='text'
    ))

    map_fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_zoom=4,
        mapbox_center={"lat": 50, "lon": 10},
        title="Distribution Centers and Delivery Delays"
    )

    
    delay_hist = px.histogram(
        filtered_data,
        x='DeliveryDelay',
        nbins=30,
        title="Distribution of Delivery Delays",
        labels={'DeliveryDelay': 'Delivery Delay (days)'}
    )

   
    quantity_line = px.line(
        filtered_data,
        x='RequestedDeliveryDate',
        y='OrderQuantity',
        color='PlantName',
        title="Order Quantity Over Time",
        labels={'RequestedDeliveryDate': 'Date', 'OrderQuantity': 'Order Quantity'}
    )

    return map_fig, delay_hist, quantity_line


if __name__ == '__main__':
    app.run_server(debug=True)

AttributeError: Can only use .str accessor with string values!

In [ ]:
# 5 

In [ ]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go


In [ ]:

data['RequestedDeliveryDate'] = pd.to_datetime(data['RequestedDeliveryDate'])
data['DeliveryDate'] = pd.to_datetime(data['DeliveryDate'])
data['SalesOrderCreationDate'] = pd.to_datetime(data['SalesOrderCreationDate'])


data['DeliveryDelay'] = data['DeliveryDelay'].astype(str)
data['DeliveryDelay'] = pd.to_numeric(data['DeliveryDelay'].str.replace(' days', ''))




In [ ]:

app = dash.Dash(__name__)


app.layout = html.Div([
    html.H1("Supply Chain Monitoring Dashboard"),
    
    html.Div([
        dcc.Dropdown(
            id='plant-type-dropdown',
            options=[{'label': plant, 'value': plant} for plant in data['PlantType'].unique()],
            value=['Distribution Center'],
            multi=True,
            placeholder="Select Plant Type"
        ),
        dcc.Dropdown(
            id='city-dropdown',
            options=[{'label': city, 'value': city} for city in data['PlantCity'].unique()],
            value=None,
            multi=True,
            placeholder="Select City"
        ),
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date=data['RequestedDeliveryDate'].min(),
            end_date=data['RequestedDeliveryDate'].max(),
            display_format='YYYY-MM-DD'
        )
    ], style={'display': 'flex', 'gap': '20px'}),
    
    dcc.Graph(id='map-plot'),
    dcc.Graph(id='delay-histogram'),
    dcc.Graph(id='quantity-line-chart')
])


@app.callback(
    [Output('map-plot', 'figure'),
     Output('delay-histogram', 'figure'),
     Output('quantity-line-chart', 'figure')],
    [Input('plant-type-dropdown', 'value'),
     Input('city-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_charts(selected_plant_types, selected_cities, start_date, end_date):
   
    filtered_data = data.copy()
    if selected_plant_types:
        filtered_data = filtered_data[filtered_data['PlantType'].isin(selected_plant_types)]
    if selected_cities:
        filtered_data = filtered_data[filtered_data['PlantCity'].isin(selected_cities)]
    if start_date and end_date:
        filtered_data = filtered_data[(filtered_data['RequestedDeliveryDate'] >= start_date) & 
                                      (filtered_data['RequestedDeliveryDate'] <= end_date)]
    
    
    map_fig = go.Figure()

   
    map_fig.add_trace(go.Scattermapbox(
        lat=filtered_data['Latitude'],
        lon=filtered_data['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=filtered_data['OrderQuantity'] / 10,
            color=filtered_data['DeliveryDelay'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Delivery Delay (days)")
        ),
        text=filtered_data.apply(lambda row: f"Plant: {row['PlantName']}<br>City: {row['PlantCity']}<br>Order Quantity: {row['OrderQuantity']}<br>Delivery Delay: {row['DeliveryDelay']} days<br>Requested Delivery Date: {row['RequestedDeliveryDate']}<br>Delivery Date: {row['DeliveryDate']}", axis=1),
        hoverinfo='text'
    ))

    map_fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_zoom=4,
        mapbox_center={"lat": 50, "lon": 10},
        title="Distribution Centers and Delivery Delays"
    )

    
    delay_hist = px.histogram(
        filtered_data,
        x='DeliveryDelay',
        nbins=30,
        title="Distribution of Delivery Delays",
        labels={'DeliveryDelay': 'Delivery Delay (days)'}
    )

 
    quantity_line = px.line(
        filtered_data,
        x='RequestedDeliveryDate',
        y='OrderQuantity',
        color='PlantName',
        title="Order Quantity Over Time",
        labels={'RequestedDeliveryDate': 'Date', 'OrderQuantity': 'Order Quantity'}
    )

    return map_fig, delay_hist, quantity_line


@app.callback(
    Output('predictive-output', 'children'),
    [Input('plant-type-dropdown', 'value'),
     Input('city-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def predictive_analytics(selected_plant_types, selected_cities, start_date, end_date):
    
    return "Predictive analytics output will be displayed here."

if __name__ == '__main__':
    app.run_server(debug=True)